# M5 - Forecasting Competition - Accuracy

Taken from competitors guide directly:

The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI). You can read more about it in [competitor's guide](../references/m5-competitors-guide.docx)

### Data Dictionary for Quick Check

File 1: “calendar.csv” 
Contains information about the dates the products are sold.
- date: The date in a “y-m-d” format.
- wm_yr_wk: The id of the week the date belongs to.
- weekday: The type of the day (Saturday, Sunday, …, Friday).
- wday: The id of the weekday, starting from Saturday.
- month: The month of the date.
- year: The year of the date.
- event_name_1: If the date includes an event, the name of this event.
- event_type_1: If the date includes an event, the type of this event.
- event_name_2: If the date includes a second event, the name of this event.
- event_type_2: If the date includes a second event, the type of this event.
- snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP  purchases on the examined date. 1 indicates that SNAP purchases are allowed.

File 2: “sell_prices.csv”
Contains information about the price of the products sold per store and date.
- store_id: The id of the store where the product is sold. 
- item_id: The id of the product.
- wm_yr_wk: The id of the week.
- sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).

File 3: “sales_train.csv” 
Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29. 


### Evaluation - Quick Check

#### Forecasting horizon

The number of forecasts required, both for point and probabilistic forecasts, is h=28 days (4 weeks ahead). 

The performance measures are **first computed for each series** separately by averaging their values across the forecasting horizon and **then averaged again across the series** in a weighted fashion (see below) to obtain the final scores.


#### Point Forecasts

The accuracy of the point forecasts will be evaluated using the Root Mean Squared Scaled Error (RMSSE), which is a variant of the well-known Mean Absolute Scaled Error (MASE) proposed by Hyndman and Koehler (2006) . The measure is calculated for each series as follows:

$$ RMSSE = \sqrt{\frac{1}{h}\frac{\sum_{t = n+1}^{n+h} (Y_{t} - \hat{Y_{t}})^{2}}{\frac{1}{n-1}\sum_{t = 2}^{n} (Y_{t} - {Y_{t-1}})^{2}}} $$

where $ Y_{t} $ is the actual future value, $ \hat{Y_{t}} $ is the predicted future value, n is the length of the training sampe and h the forecasting horizon. Intuitively it calculate how well a forecast will be regarding a naive forecasting applied to in-sample data.

*Justifications*:

 - Since we have intermittent data with a lot of zeros, a MASE approach would optimize forecast for the median, which is closer to zero. However, since major goal is to accurately forecast average demand, they choosed a metric based on squared erros, which are optimized for the mean.
 - The measure is scale independent, which is best suited for series in different scales.
 - It can be safely computed as does not rely on divisions with values that could be equal or close to zero.
 - The measure penalizes positive and negative forecast errors, as well as large and small forecasts, equally, thus being symmetric.
 
 
After estimating the RMSSE for all the 42,840 time series of the competition, the participating methods will be ranked using the Weighted RMSSE (WRMSSE), as described latter in this Guide, using the following formula:


$$ WRMSSE = \sum_{i=1}^{42,840} w_{i}*RMSSE $$

where $ w_{i} $ is the weight of the $ i_{th} $ series of the competition. A lower WRMSSE score is better.

$ \sqrt{x} $

## Importings

### Importing Libraries

In [46]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

### Importing Data

In [136]:
df_sales = pd.read_csv("../data/raw/sales_train_validation.csv")
df_sales.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
df_sample_sub = pd.read_csv("../data/raw/sample_submission.csv")
df_sample_sub.head(5)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Useful Quick Groupings

## Analyzing All Store / States

## Analyzing each State

## Basic Feature Engineering

## Baseline Models

### Really First Submission

I just want to submit data to understand how is the format and transformations necessary.

In [38]:
df_sales_melted = (df_sales.melt(id_vars=["id", "item_id", "dept_id", "store_id", "state_id", "cat_id"],
                                    var_name="date",
                                    value_name="unit_solds")
                            .assign(simple_trend=lambda df: df["date"].str.extract(r"(\d+)").astype(int))
                    )

In [47]:
categorical_cols = ["item_id", "dept_id", "store_id", "state_id", "cat_id"]
for categorical_col in categorical_cols:
    lb = LabelEncoder()
    fname = f"{categorical_col}_enc"
    df_sales_melted[fname] = lb.fit_transform(df_sales_melted[categorical_col])

In [50]:
lgmb_clf = LGBMRegressor(colsample_bytree=0.7, max_depth=7)

In [51]:
target= ['unit_solds']
features = ["simple_trend", "item_id_enc", "dept_id_enc", "store_id_enc", "state_id_enc", "cat_id_enc"]

In [52]:
lgmb_clf.fit(df_sales_melted[features], df_sales_melted[target]);

In [113]:
df_sample_sub["cat_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: x[0])
df_sample_sub["item_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[:3]))
df_sample_sub["dept_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[:2]))
df_sample_sub["store_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: '_'.join(x[3:5]))
df_sample_sub["state_id"] = df_sample_sub["id"].str.split("_").apply(lambda x: x[3])

In [114]:
df_sample_melted = (df_sample_sub
                      .melt(id_vars=["id", "item_id", "dept_id", "store_id", "state_id", "cat_id"],
                                                        var_name="date",
                                                        value_name="unit_solds")
                     .assign(simple_trend=lambda df: df["date"].str.extract(r"(\d+)").astype(int))
                    )

In [115]:
for categorical_col in categorical_cols:
    lb = LabelEncoder()
    fname = f"{categorical_col}_enc"
    lb.fit(df_sales_melted[categorical_col])
    
    df_sample_melted[fname] = lb.transform(df_sample_melted[categorical_col])

In [129]:
df_sample_melted["simple_trend"] += df_sales_melted["simple_trend"].max()

In [131]:
sample_forecast = lgmb_clf.predict(df_sample_melted[features])

In [132]:
df_sample_melted['forecast'] = sample_forecast

In [133]:
df_submission = df_sample_melted.pivot(index="id", columns="date", values="forecast")

In [134]:
df_submission.to_csv("../data/submission/firt_raw_submission.csv")

## Submission